In [1]:
import pandas as pd
import numpy as np


train = pd.read_csv("../input/train_2016_v2.csv")    # load dataset
props = pd.read_csv("../input/properties_2016.csv")
test_df = pd.read_csv("../input/sample_submission.csv")
test_df = test_df.rename(columns={'ParcelId': 'parcelid'})

C:\software_program\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
# Data preprocessing part

#The below variables are flags and lets assume if they are NA's it means the object does not exist so lets fix this
index = props.hashottuborspa.isnull()
#print(index)
props.loc[index,'hashottuborspa'] = "FALSE"

# pooltypeid10(does home have a Spa or hot tub) seems to be inconcistent with the 'hashottuborspa' field
print(props.hashottuborspa.value_counts())
print(props.pooltypeid10.value_counts())

#lets remove 'pooltypeid10' as has more missing values
# dropcols.append('pooltypeid10')

#Assume if the pooltype id is null then pool/hottub doesnt exist 
index = props.pooltypeid2.isnull()
props.loc[index,'pooltypeid2'] = 0

index = props.pooltypeid7.isnull()
props.loc[index,'pooltypeid7'] = 0

index = props.poolcnt.isnull()
props.loc[index,'poolcnt'] = 0

#Theres more missing values in the 'poolsizesum' then in 'poolcnt', 
#Let's fill in median values for poolsizesum where pool count is >0 and missing. 
#This is sensible assumption as residential pool sizes are fairly standard size in the U.S.
#Also the poolsizesum doesn't seem to be much of an important variable so imputing with the median
print(props.poolsizesum.isnull().sum())
print(props.poolcnt.value_counts())

#Fill in those properties that have a pool with median pool value
poolsizesum_median = props.loc[props['poolcnt'] > 0, 'poolsizesum'].median()
props.loc[(props['poolcnt'] > 0) & (props['poolsizesum'].isnull()), 'poolsizesum'] = poolsizesum_median

#If it doesn't have a pool then poolsizesum is 0 by default
props.loc[(props['poolcnt'] == 0), 'poolsizesum'] = 0

#There should be a one-to-one correspondence between these two items
print(props.fireplaceflag.isnull().sum())
print(props.fireplacecnt.isnull().sum())


#There seems to be 80668 properties without fireplace according to the 'fireplacecnt' but the 'fireplace flag' says they are 90053 missing values
#Lets instead create the fireplaceflag from scratch using 'fireplacecnt' as there are less missing values here
props['fireplaceflag']= "FALSE"
props.loc[props['fireplacecnt']>0,'fireplaceflag']= "TRUE"

index = props.fireplacecnt.isnull()
props.loc[index,'fireplacecnt'] = 0

#Tax deliquency flag - assume if it is null then doesn't exist
index = props.taxdelinquencyflag.isnull()
props.loc[index,'taxdelinquencyflag'] = "N"

#Same number of missing values between garage count and garage size - assume this is because when there are properties with no garages then both variables are NA
print(props.garagecarcnt.isnull().sum())
print(props.garagetotalsqft.isnull().sum())

#Assume if Null in garage count it means there are no garages
index = props.garagecarcnt.isnull()
props.loc[index,'garagecarcnt'] = 0

#Likewise no garage means the size is 0 by default
index = props.garagetotalsqft.isnull()
props.loc[index,'garagetotalsqft'] = 0

#Let's fill in some missing values using the most common value for those variables where this might be a sensible approach
#AC Type - Mostly 1's, which corresponds to central AC. Reasonable to assume most other properties are similar.
props['airconditioningtypeid'].value_counts()
index = props.airconditioningtypeid.isnull()
props.loc[index,'airconditioningtypeid'] = 1


#heating or system - Mostly 2, which corresponds to central heating 
#so seems reasonable to assume most other properties have central heating  
print(props['heatingorsystemtypeid'].value_counts())
index = props.heatingorsystemtypeid.isnull()
props.loc[index,'heatingorsystemtypeid'] = 2


print(props['threequarterbathnbr'].value_counts())
index = props.threequarterbathnbr.isnull()
props.loc[index,'threequarterbathnbr'] = 1


missingvalues_prop = (props.isnull().sum()/len(props)).reset_index()
missingvalues_prop.columns = ['field','proportion']
missingvalues_prop = missingvalues_prop.sort_values(by = 'proportion', ascending = False)
print(missingvalues_prop)
missingvaluescols = missingvalues_prop[missingvalues_prop['proportion'] > 0.90].field.tolist()
#dropcols_2 = dropcols + missingvaluescols
#props = props.drop(dropcols_2, axis=1)
props = props.drop(missingvaluescols, axis=1)

FALSE    2916203
True       69014
Name: hashottuborspa, dtype: int64
1.0    36939
Name: pooltypeid10, dtype: int64
2957257
0.0    2467683
1.0     517534
Name: poolcnt, dtype: int64
2980054
2672580
2101950
2101950
2.0     1156830
7.0      595453
6.0       27480
24.0      21107
20.0       3216
13.0       1342
18.0        586
1.0         262
14.0         41
10.0         39
12.0         25
11.0         16
21.0          3
19.0          1
Name: heatingorsystemtypeid, dtype: int64
1.0    308959
2.0      2338
3.0       261
4.0        46
5.0        16
6.0         9
7.0         2
Name: threequarterbathnbr, dtype: int64
                           field  proportion
41                   storytypeid    0.999456
3                   basementsqft    0.999455
46            yardbuildingsqft26    0.999113
2       architecturalstyletypeid    0.997970
43        typeconstructiontypeid    0.997740
13          finishedsquarefeet13    0.997430
6            buildingclasstypeid    0.995769
9                     d

In [3]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import seaborn as sns

count = props.isnull().sum()
ratio = count / len(props)
null_data = pd.concat([count,ratio],axis=1,keys=['count','ratio'])
null_data


index = []
for i, v in enumerate(ratio):
    if v > 0.5:
        index.append(i)
        
props.drop(props.columns[index], axis = 1, inplace=True)        

correlation = props.corr()
plt.subplots(figsize=(12, 12))
sns.heatmap(correlation, square=True)
plt.show()


props['citycode'] = props['propertycountylandusecode'].apply(lambda x: str(x)[0:3])
lbl = LabelEncoder()
lbl.fit(list(props['citycode'].values))
props['citycode'] = lbl.transform(list(props['citycode'].values))

props['livingareaerror'] = props['calculatedfinishedsquarefeet'] / props['finishedsquarefeet12']

props['livingareaprop'] = props['calculatedfinishedsquarefeet'] / props['lotsizesquarefeet']

props['extraspace'] = props['lotsizesquarefeet'] - props['calculatedfinishedsquarefeet']

props['life'] = 2019 - props['yearbuilt']

props['totalrooms'] = props['bathroomcnt'] + props['bedroomcnt']

props['averageroomsize'] = props['calculatedfinishedsquarefeet'] / props['roomcnt']

props['extrarooms'] = props['roomcnt'] - props['totalrooms']

props['valueproportion'] = props['structuretaxvaluedollarcnt'] / props['landtaxvaluedollarcnt']

props["location1"] = props["latitude"] + props["longitude"]

props["location2"] = props["latitude"] * props["longitude"]

props['taxratio'] = props['taxvaluedollarcnt'] / props['taxamount']

zip_count = props['regionidzip'].value_counts().to_dict()
props['zipcount'] = props['regionidzip'].map(zip_count)

city_count = props['regionidcity'].value_counts().to_dict()
props['citycount'] = props['regionidcity'].map(city_count)

region_count = props['regionidcounty'].value_counts().to_dict()
props['countycount'] = props['regionidcounty'].map(region_count)

<Figure size 1200x1200 with 2 Axes>

In [4]:
props.shape

(2985217, 56)

In [5]:
train_df = train.merge(props, how = 'left', on = 'parcelid') # get a complete dataframe
test_df = test_df.merge(props, on='parcelid', how='left')


In [6]:
train_df.shape

(90275, 58)

In [7]:
# get outlier of logerror
ulimit = np.percentile(train_df.logerror.values, 99)  
print(ulimit)
llimit = np.percentile(train_df.logerror.values, 1)
print(llimit)

0.4638819999999963
-0.3425


In [8]:
# get train data and test data without outlier
# don't change the code about test data
train_df = train_df[train_df.logerror > -0.3425]
train_df = train_df[train_df.logerror < 0.4639]
do_not_include = ['parcelid', 'logerror', 'transactiondate']
feature_names = [f for f in train_df.columns if f not in do_not_include]
#X_train = train_df.drop(['parcelid', 'logerror', 'transactiondate'] ,axis = 1)
X_train = train_df[feature_names].copy()
y_train = train_df.logerror
test_df = test_df[feature_names].copy()


In [9]:
# get the data index which is categorical 
s = (X_train.dtypes =='object')
object_cols = list(s[s].index)
print(object_cols)

['hashottuborspa', 'propertycountylandusecode', 'propertyzoningdesc', 'fireplaceflag', 'taxdelinquencyflag']


In [10]:
from sklearn.preprocessing import LabelEncoder


# use labelencoder to encoder X_train and test data
label_X_train = X_train.copy()
#label_X_valid = X_valid.copy()
test = test_df.copy()
for col in label_X_train.columns:
    label_X_train[col] = label_X_train[col].fillna(-1)
    test[col] = test[col].fillna(-1)
    if label_X_train[col].dtype =='object':
        lbl = LabelEncoder()
        lbl.fit(list(label_X_train[col].values))
        label_X_train[col] = lbl.transform(list(label_X_train[col].values))
    if  test[col].dtype == 'object':
        lbl.fit(list(test[col].values))
        test[col] = lbl.transform(list(test[col].values))

x_test = test.copy()

In [11]:
# label_X_train.head()

In [12]:
used_features = [#'calculatedfinishedsquarefeet','finishedsquarefeet12',
              # 'lotsizesquarefeet','yearbuilt','bathroomcnt','bedroomcnt'
              # ,'totalrooms','structuretaxvaluedollarcnt','landtaxvaluedollarcnt'
              # ,"latitude", "longitude",'taxvaluedollarcnt','taxamount','regionidcounty'
               # ,'regionidcity','regionidzip', 
               'averageroomsize']

In [13]:
label_X_train_sf = label_X_train.drop(used_features ,axis = 1)

x_test_sf = x_test.drop(used_features ,axis = 1)

In [14]:
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# label_X_train_sf = scaler.fit_transform(label_X_train_sf)
# x_test_sf = scaler.fit_transform(x_test_sf)

In [15]:
print(label_X_train_sf)
x_test_sf


       airconditioningtypeid  bathroomcnt  bedroomcnt  buildingqualitytypeid  \
0                        1.0          2.0         3.0                    4.0   
1                        1.0          3.5         4.0                   -1.0   
2                        1.0          3.0         2.0                    4.0   
3                        1.0          2.0         2.0                    4.0   
4                        1.0          2.5         4.0                   -1.0   
...                      ...          ...         ...                    ...   
90270                    1.0          1.0         1.0                    4.0   
90271                    1.0          3.0         3.0                    4.0   
90272                    1.0          2.0         4.0                    7.0   
90273                    1.0          2.0         2.0                    4.0   
90274                    1.0          1.0         3.0                    7.0   

       calculatedbathnbr  calculatedfin

,airconditioningtypeid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fireplacecnt,fullbathcnt,...,life,totalrooms,extrarooms,valueproportion,location1,location2,taxratio,zipcount,citycount,countycount
0,1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,6037.0,0.0,-1.0,...,-1.0,0.0,0.0,-1.000000,-84509642.0,-4.051377e+15,-1.000000,8496.0,7506.0,2009362.0
1,1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,6037.0,0.0,-1.0,...,-1.0,0.0,0.0,-1.000000,-84484934.0,-4.049921e+15,-1.000000,8496.0,7506.0,2009362.0
2,1.0,0.0,0.0,-1.0,-1.0,73026.0,-1.0,6037.0,0.0,-1.0,...,-1.0,0.0,0.0,0.853304,-84405274.0,-4.024158e+15,67.950089,9437.0,11686.0,2009362.0
3,1.0,0.0,0.0,7.0,-1.0,5068.0,-1.0,6037.0,0.0,-1.0,...,71.0,0.0,0.0,0.975846,-84288343.0,-4.044496e+15,79.466147,7916.0,670925.0,2009362.0
4,1.0,0.0,0.0,-1.0,-1.0,1776.0,-1.0,6037.0,0.0,-1.0,...,72.0,0.0,0.0,0.808511,-84191648.0,-4.048104e+15,75.716704,7725.0,670925.0,2009362.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2985212,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,...,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.000000e+00,-1.000000,-1.0,-1.0,-1.0
2985213,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,...,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.000000e+00,-1.000000,-1.0,-1.0,-1.0
2985214,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,...,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.000000e+00,-1.000000,-1.0,-1.0,-1.0
2985215,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,...,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.000000e+00,-1.000000,-1.0,-1.0,-1.0


In [16]:
# from xgboost import XGBRegressor
# from sklearn.model_selection import GridSearchCV

# parameters = {#"n_estimators": np.arange(200,500,50)
#             # , "learning_rate": np.arange(0.006, 0.02, 0.002)
#             # ,"max_depth": np.arange(6,9,1)
#              # "gamma": np.arange(0.5,2,0.3)
#              #, "reg_lambda": np.arange(0.5,2, 0.3)
#                "early_stopping_rounds": np.arange(5, 20, 2)
#               ,"reg_alpha": np.arange(0.5, 2, 0.2)
#              }

# reg =  XGBRegressor(n_estimators = 450
#                           , early_stopping_rounds = 5
#                            , learning_rate = 0.014 # 学习率不能太低
#                            , max_depth = 6
#                            , objective='reg:squarederror'
#                           # , subsample = 0.7
#                           #, reg_alpha = 0.9
#                           , reg_lambda = 1.4
#                          , gamma = 0.5)
# GS = GridSearchCV(reg, parameters, cv=3)
# GS.fit(label_X_train_sf, y_train)
# GS.best_params_
# GS.best_score_
 

In [17]:
# GS.best_params_

In [21]:
from xgboost import XGBRegressor

XGB_model_2 = XGBRegressor(n_estimators = 450
                         , early_stopping_rounds = 5
                         , learning_rate = 0.014 
                         , max_depth = 6
                         , objective='reg:squarederror'
                         , reg_alpha = 0.9
                         , reg_lambda = 1.4
                         , gamma = 0.5
                          )
# training XGBoost model with selected features data 
XGB_model_2.fit(label_X_train_sf, y_train)
#XGB_model_2.fit(label_X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, early_stopping_rounds=5,
             gamma=0.5, importance_type='gain', learning_rate=0.014,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=None,
             n_estimators=450, n_jobs=1, nthread=None,
             objective='reg:squarederror', random_state=0, reg_alpha=0.9,
             reg_lambda=1.4, scale_pos_weight=1, seed=None, silent=None,
             subsample=1, verbosity=1)

In [22]:
# get kaggle verification predictioin result
XGB_predictions_2 = XGB_model_2.predict(x_test_sf)


In [23]:
# writing csv result file
sub = pd.read_csv('../input/sample_submission.csv')
for c in sub.columns[sub.columns != 'ParcelId']:
    sub[c] = XGB_predictions_2
#     sub[c] = lasso_predictions #lasso results

print('Writing csv ...')
sub.to_csv('XGBoost_submission.csv', index=False, float_format='%.4f')

Writing csv ...
